In [1]:
import os
import glob
import pandas as pd
import numpy as np 
import csv
import re
import string
import time

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import csv
import re
import string
import emoji

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import warnings
warnings.filterwarnings("ignore")

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])
#demoji.download_codes()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
indir = "/media/manikya_varshney/My Passport/Mani(Yale)/Data/h01-20200909-20200915/h01-20200914"
outfile = "/media/manikya_varshney/My Passport/Mani(Yale)/Data/h01-20200909-20200915/(C&A)September14th.csv"

In [4]:
def Data_Cleaning(data):
    
    cols = [0, 1, 3, 4, 5,6, 7, 8, 14, 15, 16, 17, 18, 19, 22, 30, 31, 32, 34, 35, 36, 41, 42, 43, 45, 51, 52, 53, 54, 55, 56, 58, 63, 64, 65, 67, 73, 74, 75, 76, 77]
    
    data = data[data.columns.values[cols]]

    data_en = data[data['lang'] == 'en'].reset_index(drop=True)

    # RT flag

    data_en['RT_Flag'] = data_en['text'].str[:2]=='RT'

    # Date check

    #data_en['Date_Flag'] = data_en['created_at'].str[:10] == 'Tue Aug 18'

    #data_en = data_en[np.where((data_en['Date_Flag'] == True),True,False)].reset_index(drop=True)

    # Tree

    ############### is_quote_tweet --> False --> Non RT ###############

    interim = data_en[np.where((data_en['is_quote_tweet'] == False) & (data_en['RT_Flag']==False),True,False)].reset_index(drop=True)

    interim['extended_tweet_full_text'] = np.where(~interim['truncated'], interim['text'], interim['extended_tweet_full_text'])

    non_rep = interim[['created_at', 'id','user_id', 'coordinates', 'location', 'extended_tweet_full_text']]

    non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

    ############### is_quote_tweet --> False --> RT ###############

    interim = data_en[np.where((data_en['is_quote_tweet'] == False) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

    interim['RT_full_text'] = np.where(interim['RT_full_text'].isna(),interim['RT_text'],interim['RT_full_text'])

    interim = interim[['created_at', 'RT_id','RT_user_id', 'RT_coordinates', 'RT_location','RT_full_text']]

    interim = interim.drop_duplicates(['RT_id']).reset_index(drop=True)

    interim.columns = non_rep.columns

    non_rep = non_rep.append(interim,ignore_index=True)

    non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

    ############### is_quote_tweet --> True --> Non RT ###############

    interim = data_en[np.where((data_en['is_quote_tweet']== True) & (data_en['RT_Flag']== False),True,False)].reset_index(drop=True)

    interim['extended_tweet_full_text'] = np.where(~interim['truncated'],interim['text'],interim['extended_tweet_full_text'])

    interim = interim[['created_at', 'id','user_id', 'coordinates', 'location', 'extended_tweet_full_text']]

    interim.columns = non_rep.columns

    interim = interim.drop_duplicates(['id']).reset_index(drop=True)

    non_rep = non_rep.append(interim,ignore_index=True)

    non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

    interim = data_en[np.where((data_en['is_quote_tweet'] == True) & (data_en['RT_Flag']== False),True,False)].reset_index(drop=True)

    interim['QT_full_text'] = np.where(interim['QT_full_text'].isna(),interim['QT_text'],interim['QT_full_text'])

    interim = interim[['created_at','QT_id','QT_user_id', 'QT_coordinates', 'QT_location', 'QT_full_text']]

    interim = interim.drop_duplicates(['QT_id']).reset_index(drop=True)

    interim.columns = non_rep.columns

    non_rep = non_rep.append(interim,ignore_index=True)

    non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

    ############### is_quote_tweet --> True --> RT ###############

    interim = data_en[np.where((data_en['is_quote_tweet'] == True) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

    interim['RT_full_text'] = np.where(interim['RT_full_text'].isna(),interim['RT_text'],interim['RT_full_text'])
    
    interim = interim[['created_at', 'RT_id','RT_user_id', 'RT_coordinates', 'RT_location','RT_full_text']]

    interim = interim.drop_duplicates(['RT_id']).reset_index(drop=True)

    interim.columns = non_rep.columns

    non_rep = non_rep.append(interim,ignore_index=True)

    non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

    interim = data_en[np.where((data_en['is_quote_tweet']== True ) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

    interim['QT_full_text'] = np.where(interim['QT_full_text'].isna(),interim['QT_text'],interim['QT_full_text'])

    interim = interim[['created_at', 'QT_id','QT_user_id', 'QT_coordinates', 'QT_location', 'QT_full_text']]

    interim = interim.drop_duplicates(['QT_id']).reset_index(drop=True)

    interim.columns = non_rep.columns

    non_rep = non_rep.append(interim,ignore_index=True)

    non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)
    
    return non_rep

In [5]:
def Analysis(cleaned_data):
    
    #cleaned_data = non_rep.copy()

    cleaned_data['extended_tweet_full_text_duplicate'] = cleaned_data['extended_tweet_full_text']

    #Convert to lower case
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.lower()

    #Removing emojis
    def demoji(text):
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U00010000-\U0010ffff"
                                   "]+", flags=re.UNICODE)
        return(emoji_pattern.sub(r'', text))

    cleaned_data[u'extended_tweet_full_text'] = cleaned_data[u'extended_tweet_full_text'].astype(str)
    cleaned_data[u'extended_tweet_full_text'] = cleaned_data[u'extended_tweet_full_text'].apply(lambda x:demoji(x))

    #Remove URLs
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.replace(r"http\S+| www\S+| https\S+| \S+\.com\S+| \S+\.com", "", regex=True)

    #Remove user @
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.replace(r'\@[\w]+', "", regex=True)

    #Remove punctuations
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.translate(str.maketrans("", "", string.punctuation))

    #More Cleaning
    cleaned_data['extended_tweet_full_text']=cleaned_data['extended_tweet_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')

    #Tokenizing
    cleaned_data.extended_tweet_full_text = cleaned_data.extended_tweet_full_text.astype(str)
    cleaned_data['tokenized_extended_tweet_full_text'] = cleaned_data.apply(lambda row: nltk.word_tokenize(row.extended_tweet_full_text), axis=1)

    # remove stopwords
    cleaned_data['filtered_extended_tweet_full_text'] = cleaned_data['tokenized_extended_tweet_full_text'].apply(lambda x: [word for word in x if word not in stop_words])

    #Stemming
    ps = PorterStemmer()
    cleaned_data['stemmed_extended_tweet_full_text'] = cleaned_data['filtered_extended_tweet_full_text'].apply(lambda x: [ps.stem(y) for y in x])

    #POSTags
    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)

    #Lemmatizing
    lemmatizer = WordNetLemmatizer()
    cleaned_data['lemmatized_extended_tweet_full_text'] = cleaned_data['filtered_extended_tweet_full_text'].apply(lambda x: [lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in x])

    #Joining the lemmetized tokens to form string
    cleaned_data['final'] = cleaned_data['lemmatized_extended_tweet_full_text'].apply(lambda x: " ".join([word for word in x]))

    #Remove punctuations
    cleaned_data['final'] = cleaned_data['final'].str.translate(str.maketrans("", "", string.punctuation))
    cleaned_data['final'] = cleaned_data['final'].str.replace("’", '').str.replace("“", '').str.replace("”", '')
    cleaned_data['extended_tweet_full_text']=cleaned_data['extended_tweet_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')

    cols_2 = [0, 1, 2, 3 ,4 ,6, 11]
    data_final = cleaned_data[cleaned_data.columns.values[cols_2]]

    return data_final

In [6]:
def concatenate(indir, outfile):
    os.chdir(indir)
    fileList = glob.glob("*.csv")
    dfList = []
    for filename in fileList:
        print(filename)
        df = pd.read_csv(filename, header = 0, low_memory=False)
        cleaned_df = Data_Cleaning(df)
        final_df = Analysis(cleaned_df)
        dfList.append(final_df)
        print(len(dfList))
    print(len(dfList))
    concatDf = pd.concat(dfList, axis = 0)
    concatDf.to_csv(outfile, index = None)

In [7]:
start = time.time()

concatenate(indir, outfile)

end = time.time()

h01-20200914-000341.csv
1
h01-20200914-001057.csv
2
h01-20200914-001648.csv
3
h01-20200914-002242.csv
4
h01-20200914-002841.csv
5
h01-20200914-003439.csv
6
h01-20200914-004052.csv
7
h01-20200914-004703.csv
8
h01-20200914-005319.csv
9
h01-20200914-005948.csv
10
h01-20200914-011248.csv
11
h01-20200914-012607.csv
12
h01-20200914-013922.csv
13
h01-20200914-020543.csv
14
h01-20200914-023127.csv
15
h01-20200914-032329.csv
16
h01-20200914-034907.csv
17
h01-20200914-041535.csv
18
h01-20200914-043543.csv
19
h01-20200914-044224.csv
20
h01-20200914-044905.csv
21
h01-20200914-045534.csv
22
h01-20200914-050144.csv
23
h01-20200914-050759.csv


TypeError: bad operand type for unary ~: 'float'

In [ ]:
print("Total time : ",(end-start),"sec")